In [ ]:
import numpy as np
import pandas as pd

# Add parent directory to sys.path in order to find common module
import sys
sys.path.insert(0,'../')

import common.datautils as datautils
import common.flagutils as flagutils

In [ ]:
#ric_data = pd.read_csv('RIC.20180831-013501.csv')
ric_data = pd.read_csv('RIC.results.csv')
ric_data.loc[:, 'Energy'] *= 1e-6 # Convert energy to Joules
ric_data.loc[:, 'Benchmark'] = ric_data['Benchmark'].str.replace('.x', '')
ric_data.loc[:, 'Flags'] = ric_data["Flags"].str.strip()

ric_average_data = ric_data.groupby(['Benchmark','Flags', 'RunId'], as_index=False).agg({'Energy':'mean', 'Time':'mean'})
ric_average_data.head()

In [ ]:
test_configs = pd.read_csv('../scripts/random_configs_1000.csv', header=None, names=['Flags'])
test_configs.loc[:, 'Flags'] = test_configs["Flags"].str.strip()
test_configs.head()

In [ ]:
print(len(test_configs.Flags.unique()))

In [ ]:
all_flags = flagutils.load_flag_list()
print(len(all_flags))

In [ ]:
benchmarks = ric_average_data["Benchmark"].unique()
print(benchmarks)

In [ ]:
for benchmark in benchmarks:
    benchmark_data = ric_average_data[ric_average_data["Benchmark"] == benchmark]
    
    good_configs = benchmark_data["Flags"].unique()
    bad_configs = [c.split(' ') for c in test_configs["Flags"].unique() if c not in good_configs]
    print(len(good_configs))
    print(len(bad_configs))
    
    common_bad_flags = set(bad_configs[0]).intersection(*bad_configs)
    print(common_bad_flags)

In [ ]:
for benchmark in benchmarks:
    benchmark_data = ric_average_data[ric_average_data["Benchmark"] == benchmark]
    
    good_configs = benchmark_data["Flags"].unique()
    bad_configs = [c.split(' ') for c in test_configs["Flags"].unique() if c not in good_configs]
    
    flag_counts = []
    
    for flag in all_flags:
        count = 0
        for config in bad_configs:
            if (flag in config) or ("-fno-" + flag[2:] in config):
                count += 1
    
        flag_counts.append((flag, count))
        
    flag_counts = sorted(flag_counts, key=lambda x: x[1])
    print(benchmark)
    print(flag_counts[0:5])
    print("")